In [3]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import pandas as pd
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

In [12]:
df = pd.read_csv("./data/reddit/cm/askanincel_comments.csv")

In [186]:
sdf = pd.read_csv('subreddits.csv')
subreddits = sdf.values.tolist()

In [191]:
for s in subreddits[1:]:
    sub = str(s)[5:-5]
    print('sub', sub)     

    temp_df = pd.read_csv(f'./data/reddit/cm/{sub}_comments.csv')
    temp_serie = temp_df.Comment
    if s == subreddits[1]:
        serie = temp_serie
    else:
        serie = serie.append(temp_serie, ignore_index=True)

sub exredpill
sub RedPillParenting
sub redpillbooks
sub TheRedPill
sub RedPillWomen
sub asktrp
sub thankTRP
sub becomeaman
sub GEOTRP
sub TRPOffTopic
sub Braincels
sub askanincel
sub BlackPillScience
sub IncelsWithoutHate
sub ForeverAlone
sub MensRightsLaw
sub MRActivism
sub FeMRA
sub LadyMRAs
sub Masculism
sub MensRants
sub MRRef
sub FeMRADebates
sub againstmensrights
sub TheBluePill


In [192]:
serie.head()

0    Did many other people at your school/college d...
1                                            [deleted]
2    Seconding this. What do you think it would tak...
3    Personally, I think the best way redpillers co...
4                                            [deleted]
Name: Comment, dtype: object

In [193]:
serie.shape

(9061164,)

In [195]:
serie = serie[serie != '[deleted]']

In [197]:
serie.shape

(8501763,)

## Pre-processing data

In [198]:
nlp = spacy.load("en")

In [199]:
my_stop_words = [u'say', u'\s', u'Mr', u'be', u'said', u'says', u'saying', u's', u'’s', u'\n\n']
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [200]:
pre_doc = pre_doc.replace('/', ' ')

In [201]:
stl= pd.Series.tolist(serie)

In [203]:
texts, article, skl_texts = [], [], []
for comment in stl:
    doc = nlp(str(comment))
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)
        # if it's a new line, it means we're onto our next document
        if w.text == doc[-1].text:
            skl_texts.append(' '.join(article))
            texts.append(article)
            article = []

KeyboardInterrupt: 

In [ ]:
texts

In [ ]:
bigram = gensim.models.Phrases(texts)

In [ ]:
texts = [bigram[line] for line in texts]

In [ ]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
corpus[5][0:10]

## LSI

In [ ]:
lsimodel = LsiModel(corpus=corpus, num_topics=20, id2word=dictionary)

In [ ]:
lsimodel.show_topics()  # Showing only the top 5 topics

## HDP

In [164]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)

In [168]:
hdpmodel.show_topics(num_topics=5)

[(0,
  '0.037*\n\n + 0.020*woman + 0.012*like + 0.012*man + 0.012*incel + 0.010*people + 0.009*not + 0.008*think + 0.008*want + 0.008*  + 0.007*look + 0.007*know + 0.006*guy + 0.006*thing + 0.005*girl + 0.005*good + 0.004*\n + 0.004*way + 0.004*get + 0.004*try'),
 (1,
  '0.018*\n\n + 0.007*woman + 0.006*like + 0.005*incel + 0.005*not + 0.004*people + 0.004*think + 0.004*want + 0.004*man + 0.003*know + 0.003*be + 0.002*look + 0.002*guy + 0.002*thing + 0.002*’s + 0.002*try + 0.002*girl + 0.002*way + 0.002*ask + 0.002* '),
 (2,
  '0.014*\n\n + 0.006*woman + 0.004*like + 0.004*incel + 0.004*man + 0.003*people + 0.003*not + 0.003*\n + 0.003*think + 0.002*want + 0.002*look + 0.002*know + 0.002*guy + 0.002*  + 0.002*be + 0.002*girl + 0.002*thing + 0.002*good + 0.002*’s + 0.002*try'),
 (3,
  '0.013*\n\n + 0.006*woman + 0.005*like + 0.004*incel + 0.004*people + 0.003*man + 0.003*think + 0.003*not + 0.003*want + 0.002*look + 0.002*  + 0.002*thing + 0.002*know + 0.002*guy + 0.002*try + 0.002*\n +

## LDA

In [183]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)

In [184]:
ldamodel.show_topics(num_topics=5)

[(0,
  '0.020*"incel" + 0.019*"woman" + 0.016*"\n\n" + 0.011*"date" + 0.011*"think" + 0.010*"foid" + 0.010*"guy" + 0.009*"look" + 0.009*"like" + 0.007*"man"'),
 (1,
  '0.025*"yes" + 0.017*"\n\n" + 0.016*"\n" + 0.013*"face" + 0.010*"white" + 0.010*"work" + 0.009*"guess" + 0.008*"woman" + 0.006*"short" + 0.006*"tall"'),
 (2,
  '0.029*"people" + 0.025*"like" + 0.018*" " + 0.016*"want" + 0.015*"look" + 0.015*"try" + 0.015*"know" + 0.014*"thing" + 0.014*"\n\n" + 0.014*"good"'),
 (3,
  '0.046*"woman" + 0.033*"man" + 0.022*"\n\n" + 0.014*"think" + 0.012*"’s" + 0.011*"want" + 0.011*"not" + 0.011*"like" + 0.011*"be" + 0.010*"girl"'),
 (4,
  '0.094*"\n\n" + 0.031*"incel" + 0.021*"woman" + 0.016*"hate" + 0.016*"gt" + 0.014*"fuck" + 0.012*"female" + 0.012*"man" + 0.010*"\n\n_amp#x200B" + 0.009*"lol"')]

## pyLDAvis

In [185]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

KeyboardInterrupt: 

## Topic Coherence

In [ ]:
lsitopics = [[word for word, prob in topic] for topicid, topic in lsimodel.show_topics(formatted=False)]

hdptopics = [[word for word, prob in topic] for topicid, topic in hdpmodel.show_topics(formatted=False)]

ldatopics = [[word for word, prob in topic] for topicid, topic in ldamodel.show_topics(formatted=False)]

In [ ]:
lsi_coherence = CoherenceModel(topics=lsitopics[:10], texts=texts, dictionary=dictionary, window_size=10).get_coherence()

hdp_coherence = CoherenceModel(topics=hdptopics[:10], texts=texts, dictionary=dictionary, window_size=10).get_coherence()

lda_coherence = CoherenceModel(topics=ldatopics, texts=texts, dictionary=dictionary, window_size=10).get_coherence()

In [ ]:
def evaluate_bar_graph(coherences, indices):
    """
    Function to plot bar graph.
    
    coherences: list of coherence values
    indices: Indices to be used to mark bars. Length of this and coherences should be equal.
    """
    assert len(coherences) == len(indices)
    n = len(coherences)
    x = np.arange(n)
    plt.bar(x, coherences, width=0.2, tick_label=indices, align='center')
    plt.xlabel('Models')
    plt.ylabel('Coherence Value')

In [ ]:
evaluate_bar_graph([lsi_coherence, hdp_coherence, lda_coherence],
                   ['LSI', 'HDP', 'LDA'])